In [ ]:
import numpy as np
from sympy import Matrix, symbols, Poly, invert, gcd
from sympy.polys.matrices import DomainMatrix
from numpy import linalg as la

In [ ]:
def poly_mult_mod(a, b, N, q):
    """Multiplies two polynomials a and b modulo q and X^N - 1"""

    result = np.zeros(N, dtype=int)
    for i in range(N):
        for j in range(N):
            result[(i + j) % N] += (a[i] * b[j])
    #print("Print polynomial Multiplication: ",result%q)
    return result%q

def poly_inv_mod(f, N, q):
    """Finds the inverse of a polynomial f mod q using SymPy's invert function."""
    x = symbols('x')

    f_poly = Poly(f, x).set_modulus(q)
    #print(f_poly)
    modulus_poly = Poly(x**N - 1, x).set_modulus(q)

    try:
        # Check if the polynomial is invertible (gcd(f, x^N - 1) should be 1)
        if gcd(f_poly, modulus_poly) != 1:
            print(f"Polynomial {f} is not invertible mod {q}")
            return None

        f_inv_poly = invert(f_poly, modulus_poly)  # Finds the modular inverse
        f_inv = np.array(f_inv_poly.all_coeffs(), dtype=int) % q  # Convert to NumPy array


        # Ensure the resulting inverse has N coefficients
        if len(f_inv) < N:
            f_inv = np.concatenate([np.zeros(N - len(f_inv)), f_inv])  # Pad with zeros
        return f_inv
    except Exception as e:
        print(f"Error computing inverse for polynomial {f}: {e}")
        return None  # Return None if the inverse doesn't exist

In [ ]:
def gen_random_poly(d1, d2, N):
    """Generates a random ternary polynomial with exactly d1 coefficients equal to 1 and d2 equal to -1."""
    poly = [0] * N
    ones = np.random.choice(N, d1, replace=False)  # Select positions for +1s
    minus_ones = np.random.choice(list(set(range(N)) - set(ones)), d2, replace=False)  # Select positions for -1s

    for i in ones:
        poly[i] = 1
    for i in minus_ones:
        poly[i] = -1

    return poly

def keygen(N, p, q):
    """Key generation: creates the public key h and private keys f, Fp, Fq"""
    while True:
        # Randomly choose f from T(d+1, d) and g from T(d, d)
        #f=[-1,0,1,1,-1,0,1]
        f = gen_random_poly(d1=N//3+1, d2=N//3, N=N)
        g = gen_random_poly(d1=N//3, d2=N//3, N=N)
        print("f: ",f)
        print("g: ",g)
        buffer_f=f
        buffer_f.reverse()
        #g=[0,-1,-1,0,1,0,1]

        Fq = poly_inv_mod(buffer_f, N, q)
        Fq=np.flip(Fq)
        print("Inverse of f mod q (Fq):", Fq)
        Fp = poly_inv_mod(buffer_f, N, p)
        Fp=np.flip(Fp)
        print("Inverse of f mod p (Fp):", Fp)

        if Fq is not None and Fp is not None:
            break

    h = p*poly_mult_mod(Fq, g, N, q)%q
    f.reverse()

    return f, Fp, Fq, h,g

In [ ]:
def encrypt(h, m, r, N, p, q):
    """Encrypts message m using public key h and random r"""


    pr = poly_mult_mod(h, r, N, q)  # p * r
    e = (pr + m) % q  # e = p*r + h*m mod q
    return e

def decrypt(e, f, Fp, N, p, q):
  """Decrypts encrypted message e using private keys f and Fp"""
  a = poly_mult_mod(f, e, N, q)  # Multiply f and e mod q
  a = np.array([(x if x <= q // 2 else x - q) for x in a])  # Coefficients in [-q/2, q/2]

  # Multiply Fp and a mod p
  m_recovered = poly_mult_mod(Fp, a, N, p) % p
  # Ensure m_recovered values are in [-p/2, p/2]
  m_recovered = np.array([(x if x <= p // 2 else x - p) for x in m_recovered])
  return m_recovered

In [ ]:
N = 7   # Degree of the polynomials (choose small for example)
p = 3   # Small modulus
q = 41  # Large modulus

# Key generation
f, Fp, Fq, h,g = keygen(N, p, q)
print("Public key (h):", h)
print("Private key (f):", f)

# Encryption
m = np.array([1, 0, 1, 1, 0, -1, 0])  # Example message as a polynomial
r = gen_random_poly(d1=N//3, d2=N//3, N=N)  # Random polynomial r
print("r: ",r)
#r=[-1,1,0,0,0,-1,1]
e = encrypt(h, m, r, N, p, q)
print("Encrypted message (e):", e)


# Decryption
m_recovered = decrypt(e, f, Fp, N, p, q)
print("Decrypted message (m_recovered):", m_recovered)
print("Original message: ",m)


f:  [-1, 1, 1, 0, -1, 0, 1]
g:  [0, 0, 1, -1, 0, -1, 1]
Inverse of f mod q (Fq): [ 4 38 25 13 23 30 32]
Inverse of f mod p (Fp): [2 0 1 1 2 2 2]
Public key (h): [19  1  9 40 37  7 10]
Private key (f): [-1, 1, 1, 0, -1, 0, 1]
r:  [0, 1, 1, -1, -1, 0, 0]
Encrypted message (e): [23 26  4 23 29 25 36]
Decrypted message (m_recovered): [ 1  0  1  1  0 -1  0]
Original message:  [ 1  0  1  1  0 -1  0]


In [ ]:
g

[0, 0, 1, -1, 0, -1, 1]

In [ ]:
def create_lattice_matrix(h, N, q):
    H = np.zeros((N, N), dtype=int)
    for i in range(N):
        H[i] = np.roll(h, i)
    I = np.eye(N, dtype=int)
    M_upper = np.hstack((I, H))
    M_lower = np.hstack((np.zeros((N, N), dtype=int), q * I))
    return np.vstack((M_upper, M_lower))

In [ ]:
def create_scaled_lattice_matrix(h, f, g, N, q):
    # Calculate scaling factor `a` based on norm ratios of f and g
    norm_f = la.norm(f)
    norm_g = la.norm(g)
    a = int(np.round((norm_g ** 2) / (norm_f ** 2)))  # Optimal scaling factor

    # Construct H (circulant matrix for h)
    H = np.zeros((N, N), dtype=int)
    for i in range(N):
        H[i] = np.roll(h, i)
    I = np.eye(N, dtype=int)

    # Add scaled rows with af and g for the attack lattice
    af = (a * np.array(f)) % q
    attack_vector = np.concatenate([af, g])

    # Construct lattice matrix
    M_upper = np.hstack((I, H))
    M_lower = np.hstack((np.zeros((N, N), dtype=int), q * I))
    lattice_matrix = np.vstack((M_upper, M_lower))
    #lattice_matrix = np.vstack((M_upper, M_lower, attack_vector))

    print("Scaling factor (a):", a)
    return lattice_matrix

In [ ]:
lattice_matrix = create_scaled_lattice_matrix(h, f, g, N, q)
print("Lattice Matrix:")
print(lattice_matrix)

Scaling factor (a): 1
Lattice Matrix:
[[ 1  0  0  0  0  0  0 19  1  9 40 37  7 10]
 [ 0  1  0  0  0  0  0 10 19  1  9 40 37  7]
 [ 0  0  1  0  0  0  0  7 10 19  1  9 40 37]
 [ 0  0  0  1  0  0  0 37  7 10 19  1  9 40]
 [ 0  0  0  0  1  0  0 40 37  7 10 19  1  9]
 [ 0  0  0  0  0  1  0  9 40 37  7 10 19  1]
 [ 0  0  0  0  0  0  1  1  9 40 37  7 10 19]
 [ 0  0  0  0  0  0  0 41  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0 41  0  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0 41  0  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0 41  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0 41  0  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0 41  0]
 [ 0  0  0  0  0  0  0  0  0  0  0  0  0 41]]


In [ ]:
h

array([19,  1,  9, 40, 37,  7, 10])

In [ ]:
len(lattice_matrix)

NameError: name 'lattice_matrix' is not defined

In [ ]:
b = np.array(create_lattice_matrix(h,N,q)).astype(float)
ans=Matrix(np.array(create_lattice_matrix(h,N,q)))
print(b)

b_star = b.copy()   # Initialize the Gram-Schmidt basis.

k = 1  # Initialize the working index.
delta = 0.75  # Lovasz constant

def mu(u, v):
    '''Computes <u,v>/<u,u>, which is the scale used in projection.'''
    return np.dot(u, v) / np.dot(u, u)

def proj(u, v):
    '''Computes the projection of vector v onto vector u. Assumes u is not zero.'''
    return mu(u, v) * u

def gram_schmidt():
    '''Computes Gram Schmidt orthogonalization of a basis.'''
    b_star[0] = b[0]
    for i in range(1, b.shape[0]):  # Loop through dimension of basis.
        b_star[i] = b[i]
        for j in range(0, i):
            b_star[i] -= proj(b_star[j], b[i])
    # b_star

def reduction():
    '''Performs length reduction on a basis.'''
    global k
    total_reduction = 0  # Track the total amount by which the working vector is reduced.
    for j in range(k-1, -1, -1):   # j loop. Loop down from k-1 to 0.
        m = round(mu(b_star[j], b[k]))
        total_reduction += m * b[j][0]
        b[k] -= m * b[j]  # Reduce the working vector by multiples of preceding vectors.
    if total_reduction > 0:
        gram_schmidt()  # Recompute Gram-Schmidt if the working vector has been reduced.

def lovasz():
    global k
    '''Checks the Lovasz condition for a basis. Either swaps adjacent basis vectors and recomputes Gram-Schmidt or increments the working index.'''
    c = delta - mu(b_star[k-1], b[k])**2
    if la.norm(b_star[k])**2 >= (c * la.norm(b_star[k-1])**2):  # Check the Lovasz condition.
        k += 1  # Increment k if the condition is met.
    else:
        b[[k, k-1]] = b[[k-1, k]]  # Swap the working vector and the immediately preceding basis vector.
        gram_schmidt()  # Recompute Gram-Schmidt if swapped.
        k = max([k-1, 1])

def lll():
    global k
    gram_schmidt()
    steps = 0
    while k <= b.shape[0] - 1:
        reduction()
        steps += 1


        lovasz()
        steps += 1


    print('LLL Reduced Basis:\n', b)
    print('No of. steps in calculation: ', steps)

# print("b_star: ", b_star)
lll()




[[ 1.  0.  0.  0.  0.  0.  0. 19.  1.  9. 40. 37.  7. 10.]
 [ 0.  1.  0.  0.  0.  0.  0. 10. 19.  1.  9. 40. 37.  7.]
 [ 0.  0.  1.  0.  0.  0.  0.  7. 10. 19.  1.  9. 40. 37.]
 [ 0.  0.  0.  1.  0.  0.  0. 37.  7. 10. 19.  1.  9. 40.]
 [ 0.  0.  0.  0.  1.  0.  0. 40. 37.  7. 10. 19.  1.  9.]
 [ 0.  0.  0.  0.  0.  1.  0.  9. 40. 37.  7. 10. 19.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  9. 40. 37.  7. 10. 19.]
 [ 0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. 41.]]
LLL Reduced Basis:
 [[ 1.  1.  1.  1.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0. -1.  1. -1. -1.  0.  1.  3. -3.  0.  0. -3.  3.  0.]
 [ 1.  0. -1.  1. -1. -1.  0.  0.  

In [ ]:
b

array([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.],
       [ 0., -1.,  1., -1., -1.,  0.,  1.,  3., -3.,  0.,  0., -3.,  3.,
         0.],
       [ 1.,  0., -1.,  1., -1., -1.,  0.,  0.,  3., -3.,  0.,  0., -3.,
         3.],
       [ 0.,  0., -2., -1.,  1.,  1., -1., -3.,  0., -3.,  0.,  3.,  3.,
         0.],
       [ 0.,  1.,  0., -1.,  1., -1., -1.,  3.,  0.,  3., -3.,  0.,  0.,
        -3.],
       [ 1.,  1.,  0., -1.,  0.,  1., -1.,  0.,  3., -3.,  0., -3.,  3.,
         0.],
       [-1.,  1., -1., -1.,  0.,  1.,  0., -3.,  0.,  0., -3.,  3.,  0.,
         3.],
       [-5.,  1.,  5., -2.,  5., -4.,  1.,  0.,  2., -1., -1.,  1.,  0.,
        -1.],
       [ 0., -2.,  3.,  0.,  1., -5.,  1., -3.,  2., -1., -3.,  5.,  3.,
        -3.],
       [ 4.,  4., -6., -3., -3., -1.,  3.,  1., -2., -1.,  2.,  0., -1.,
         1.],
       [-3.,  0.,  3., -5., -2.,  1.,  5.,  0.,  3., -2.,  1.,  0., -2.,
         0.],
       [-5., -2.,  1.,  5., -3.,  0.,  3., 

In [ ]:
from sympy.polys.domains import ZZ, QQ

In [ ]:
dM = DomainMatrix.from_Matrix(ans)
dM.lll().to_Matrix()

Matrix([
[ 1,  1,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0],
[ 0, -1,  1, -1, -1,  0,  1,  3, -3,  0,  0, -3,  3,  0],
[ 1,  0, -1,  1, -1, -1,  0,  0,  3, -3,  0,  0, -3,  3],
[ 0,  0, -2, -1,  1,  1, -1, -3,  0, -3,  0,  3,  3,  0],
[ 0,  1,  0, -1,  1, -1, -1,  3,  0,  3, -3,  0,  0, -3],
[ 1,  1,  0, -1,  0,  1, -1,  0,  3, -3,  0, -3,  3,  0],
[-1,  1, -1, -1,  0,  1,  0, -3,  0,  0, -3,  3,  0,  3],
[-5,  1,  5, -2,  5, -4,  1,  0,  2, -1, -1,  1,  0, -1],
[ 5, -3,  0,  3, -5, -2,  1,  0,  0,  3, -2,  1,  0, -2],
[ 4,  4, -6, -3, -3, -1,  3,  1, -2, -1,  2,  0, -1,  1],
[-3,  0,  3, -5, -2,  1,  5,  0,  3, -2,  1,  0, -2,  0],
[-5, -2,  1,  5, -3,  0,  3,  1,  0, -2,  0,  0,  3, -2],
[-6,  4,  3,  2,  0, -2, -4, -1, -2, -2,  3,  1,  2, -1],
[ 0, -4,  2,  3,  1, -4,  0,  7,  6,  5,  5,  4,  7,  7]])

In [ ]:
from itertools import product


In [ ]:
def find_f_combination(matrix, f, N):
    """
    Check if a linear combination of the first N elements of each row in the matrix
    can form the target vector f.

    Parameters:
        matrix (np.array): LLL-reduced basis matrix.
        f (list or np.array): Target private key vector.
        N (int): Length of each subvector.

    Returns:
        list: A list of coefficient sets that form f if found.
    """
    # Extract the first N elements from each row
    subvectors = [row[:N] for row in matrix]
    target_f = np.array(f[:N])  # Target subvector for comparison

    # Generate coefficient combinations of -1, 0, and 1 for each subvector
    coefficients = list(product([-1, 0, 1], repeat=len(subvectors)))
    successful_combinations = []

    # Check each coefficient combination
    for coeff_set in coefficients:
        # Linear combination using the current set of coefficients
        combined_vector = sum(c * sub for c, sub in zip(coeff_set, subvectors))

        # Check if the combined vector matches target_f
        if np.array_equal(combined_vector, target_f):
            successful_combinations.append(coeff_set)
            print(f"Match found with coefficients: {coeff_set}")

    if not successful_combinations:
        print("No combination found that matches the private key f.")
    return successful_combinations

In [ ]:
combinations = find_f_combination(b, f, N)

Match found with coefficients: (0, 0, 0, -1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0)
